In [1]:
%load_ext autoreload
%autoreload 2
import os
import shutil

In [2]:
from CrocoDash.grid import Grid

grid = Grid.from_supergrid("/glade/derecho/scratch/manishrv/croc_input/gustavo-glorys-1/ocnice/ocean_hgrid.nc")


In [3]:
from CrocoDash.topo import Topo

bathymetry_path='/glade/derecho/scratch/manishrv/croc_input/gustavo-glorys-1/ocnice/ocean_topo.nc'

topo = Topo.from_topo_file(
    grid = grid,
    topo_file_path=bathymetry_path,
    min_depth = 9.5,
)

In [4]:
from CrocoDash.vgrid import VGrid

vgrid_path='/glade/derecho/scratch/manishrv/croc_input/gustavo-glorys-1/ocnice/vgrid_75_2m.nc'

vgrid  = VGrid.from_file(vgrid_path)


In [5]:
from pathlib import Path

In [6]:
# CESM case (experiment) name
casename = "nwa.jra.1"

# CESM source root (Update this path accordingly!!!)
cesmroot = Path("/glade/u/home/manishrv/work/installs/CROCESM_beta04_clean")

# Place where all your input files go 
inputdir = Path.home() / "scratch" / "croc_input" / casename
    
# CESM case directory
caseroot = Path.home() / "croc_cases" / casename

In [7]:
from CrocoDash.case import Case
grid.name = "nwa_jra_1"
case = Case(
    cesmroot = cesmroot,
    caseroot = caseroot,
    inputdir = inputdir,
    ocn_grid = grid,
    ocn_vgrid = vgrid,
    ocn_topo = topo,
    project = 'NCGD0011',
    override = True,
    machine = "derecho",
    compset = "1850_DATM%JRA_SLND_SICE_MOM6_DROF%JRA_SGLC_SWAV"
)

INFO:  csp_solver:CspSolver initialized.


Creating case...

• Updating ccs_config/modelgrid_aliases_nuopc.xml file to include the new resolution "nwa.jra.1" consisting of the following component grids.
 atm grid: "TL319", lnd grid: "TL319", ocn grid: "nwa_jra_1".

• Updating ccs_config/component_grids_nuopc.xml file to include newly generated ocean grid "nwa_jra_1" with the following properties:
 nx: 740, ny: 780. ocean mesh: /glade/u/home/manishrv/scratch/croc_input/nwa.jra.1/ocnice/ESMF_mesh_nwa_jra_1_74f9c7.nc.

Running the create_newcase tool with the following command:

/glade/u/home/manishrv/work/installs/CROCESM_beta04_clean/cime/scripts/create_newcase --compset 1850_DATM%JRA_SLND_SICE_MOM6_DROF%JRA_SGLC_SWAV --res nwa.jra.1 --case /glade/u/home/manishrv/croc_cases/nwa.jra.1 --machine derecho --run-unsupported --project NCGD0011 --non-local 

The create_newcase command was successful.

Navigating to the case directory:

cd /glade/u/home/manishrv/croc_cases/nwa.jra.1

Apply NTASK grid xml changes:

./xmlchange NTASKS_OCN

INFO:	stage:SUCCESS: All stages are complete.


Case created successfully at /glade/u/home/manishrv/croc_cases/nwa.jra.1.

To further customize, build, and run the case, navigate to the case directory in your terminal. To create another case, restart the notebook.



In [8]:
case.configure_forcings(
    date_range = ["2000-01-01 00:00:00", "2019-12-31 00:00:00"],
    tidal_constituents = ['M2','S2','N2','K2','K1','O1','P1','Q1','MM','MF'],
    tpxo_elevation_filepath = "/glade/u/home/manishrv/scratch/ice/croc/tpxo/h_tpxo9.v1.nc",
    tpxo_velocity_filepath = "/glade/u/home/manishrv/scratch/ice/croc/tpxo/u_tpxo9.v1.nc",
    too_much_data = True, 
    boundaries = ["south", "north", "east"]
)

INFO:CrocoDash.raw_data_access.datasets.glorys:This data access method retuns a script at path /glade/derecho/scratch/manishrv/tmp/tmp97fedwxa/get_glorys_data.sh to run to get access data 


Large data workflow was called, please go to the large data workflow path: /glade/u/home/manishrv/scratch/croc_input/nwa.jra.1/glorys/large_data_workflow and run the driver script there.


In [9]:
case.process_forcings(process_initial_condition = False, process_chl = False,process_runoff= False, process_tides=False, process_velocity_tracers=False, process_param_changes=True)

Adding parameter changes to user_nl_mom:

  ! Initial conditions
  INIT_LAYERS_FROM_Z_FILE = True
  TEMP_SALT_Z_INIT_FILE = init_tracers.nc
  Z_INIT_FILE_PTEMP_VAR = temp
  Z_INIT_ALE_REMAPPING = True
  TEMP_SALT_INIT_VERTICAL_REMAP_ONLY = True
  DEPRESS_INITIAL_SURFACE = True
  SURFACE_HEIGHT_IC_FILE = init_eta.nc
  SURFACE_HEIGHT_IC_VAR = eta_t
  VELOCITY_CONFIG = file
  VELOCITY_FILE = init_vel.nc

  ! Tides
  TIDES = True
  TIDE_M2 = True
  CD_TIDES = 0.0018
  TIDE_USE_EQ_PHASE = True
  TIDE_REF_DATE = 2000, 1, 1
  OBC_TIDE_ADD_EQ_PHASE = True
  OBC_TIDE_N_CONSTITUENTS = 10
  OBC_TIDE_CONSTITUENTS = "M2, S2, N2, K2, K1, O1, P1, Q1, MM, MF"
  OBC_TIDE_REF_DATE = 2000, 1, 1

  ! Open boundary conditions
  OBC_NUMBER_OF_SEGMENTS = 3
  OBC_FREESLIP_VORTICITY = False
  OBC_FREESLIP_STRAIN = False
  OBC_COMPUTED_VORTICITY = True
  OBC_COMPUTED_STRAIN = True
  OBC_ZERO_BIHARMONIC = True
  OBC_TRACER_RESERVOIR_LENGTH_SCALE_OUT = 3.0E+04
  OBC_TRACER_RESERVOIR_LENGTH_SCALE_IN = 3000.0
  BRU

# Post Setup
1. Delete the user_nl_mom changes
2. Copy the MOM_input Gustavo has
3. Copy the datm_streams.xml

In [10]:

input_file = caseroot/"user_nl_mom"
output_file = caseroot/"user_nl_mom_filtered"

# Keywords to keep
keywords = ("ALE_COORDINATE_CONFIG", "GRID_FILE", "TOPO_FILE", "INPUTDIR")

# Read and filter the lines
with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        if line.strip().startswith(keywords):
            outfile.write(line)
    # Add the new line at the end
    outfile.write("USE_CFC_CAP=False\n")

os.rename(output_file, input_file)

In [11]:
source_file = "/glade/u/home/manishrv/croc_cases/gustavo-glorys-2/SourceMods/src.mom/MOM_input"
destination_file = caseroot/"SourceMods/src.mom/MOM_input"
shutil.copy(source_file, destination_file)

PosixPath('/glade/u/home/manishrv/croc_cases/nwa.jra.1/SourceMods/src.mom/MOM_input')

In [12]:
source_file = "/glade/u/home/manishrv/croc_cases/gustavo-glorys-2/datm.streams.xml"
destination_file = caseroot/"datm.streams.xml"
shutil.copy(source_file, destination_file)

PosixPath('/glade/u/home/manishrv/croc_cases/nwa.jra.1/datm.streams.xml')

In [13]:
source_file = "/glade/u/home/manishrv/croc_cases/nwa.glofas.2/drof.streams.xml"
destination_file = caseroot/"drof.streams.xml"
shutil.copy(source_file, destination_file)

PosixPath('/glade/u/home/manishrv/croc_cases/nwa.glofas.3/drof.streams.xml')

In [13]:
source_file = "/glade/derecho/scratch/manishrv/croc_input/gustavo-glorys-1/ocnice/seawifs-clim-1997-2010-nwa12.nc"
destination_file = inputdir/"ocnice/seawifs-clim-1997-2010-nwa12.nc"
shutil.copy(source_file, destination_file)

PosixPath('/glade/u/home/manishrv/scratch/croc_input/nwa.jra.1/ocnice/seawifs-clim-1997-2010-nwa12.nc')

In [14]:
from visualCaseGen.custom_widget_types.case_tools import xmlchange
xmlchange("CALENDAR", "GREGORIAN", is_non_local=True)

./xmlchange CALENDAR=GREGORIAN --non-local

